In [33]:
%pip install pandas scikit-learn nltk tqdm



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re

import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

tqdm.pandas()

In [35]:
df = pd.read_csv('spam_Emails_data.csv')

df.head()

,label,text
0,Spam,viiiiiiagraaaa\nonly for the ones that want to...
1,Ham,got ice thought look az original message ice o...
2,Spam,yo ur wom an ne eds an escapenumber in ch ma n...
3,Spam,start increasing your odds of success & live s...
4,Ham,author jra date escapenumber escapenumber esca...


### Data exploration

### Preprocessing

In [ ]:
target = df['label']

label = LabelEncoder()
target = label.fit_transform(target)
label_names=['Spam','Ham'] # 0 -> Ham, 1 -> Spam
label_order=label.transform(label_names)

target[:10]

array([1, 0, 1, 1, 0, 1, 0, 0, 0, 0])

In [ ]:
# required NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/I568645/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/I568645/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/I568645/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/I568645/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Lowercase, Replace non-alphamumeric, Tokenize, Lemmatize

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text: str):
    """Lowercase, remove non-alphanumeric chars, tokenize, remove stopwords, lemmatize.
    """
    if not isinstance(text, str):
        return ""
    text = text.lower()
    # eeplace non-alphanumeric characters with space
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    # tokenize
    tokens = nltk.word_tokenize(text)
    # remove stopwords & very short tokens
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]
    # lemmatize
    lemmas = [lemmatizer.lemmatize(t) for t in tokens]
    return ' '.join(lemmas), lemmas


result_series = df['text'].progress_apply(preprocess)
df[['clean_text', 'tokens']] = result_series.apply(pd.Series)

df[['text','clean_text', 'tokens']].head()


100%|██████████| 193852/193852 [01:51<00:00, 1733.54it/s]


,text,clean_text,tokens
0,viiiiiiagraaaa\nonly for the ones that want to...,viiiiiiagraaaa one want make scream prodigy sc...,"[viiiiiiagraaaa, one, want, make, scream, prod..."
1,got ice thought look az original message ice o...,got ice thought look original message ice oper...,"[got, ice, thought, look, original, message, i..."
2,yo ur wom an ne eds an escapenumber in ch ma n...,wom ed escapenumber arn ific ati ons escapelon...,"[wom, ed, escapenumber, arn, ific, ati, ons, e..."
3,start increasing your odds of success & live s...,start increasing odds success live sexually he...,"[start, increasing, odds, success, live, sexua..."
4,author jra date escapenumber escapenumber esca...,author jra date escapenumber escapenumber esca...,"[author, jra, date, escapenumber, escapenumber..."


#### Clean potential empty rows
stop word removal could lead to empty mails

In [ ]:
# remove rows where clean_text is empty or tokens is an empty list
mask = (
    df['clean_text'].notna() &
    df['tokens'].notna() &
    (df['clean_text'].str.strip() != "") &
    df['tokens'].apply(lambda t: isinstance(t, list) and len(t) > 0)
)
removed_count = len(df) - mask.sum()
df = df[mask].copy()

# re-align target with filtered df
target = label.transform(df['label'])

print(f"Rows removed: {removed_count}; Remaining: {len(df)}")

Rows removed: 262; Remaining: 193590


#### Train-Test Split

In [43]:
data = df['tokens']
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.2, random_state=42)


#### Feature Extraction (TF-IDF and Ngrams)

In [ ]:
# dummy tokenizer (to use pre-tokenized input)
def identity_tokenizer(text):
    return text

vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    preprocessor=identity_tokenizer, # Skip preprocessing
    tokenizer=identity_tokenizer      # Use the pre-tokenized list
)

# create tf-idf vectors
X_tfidf = vectorizer.fit_transform(data_train)
X_test_tfidf = vectorizer.transform(data_test)

/Users/I568645/git/ANLP-Mailsense/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


#### Train and Test with MNB

In [ ]:
mnb_model = MultinomialNB()

# train
mnb_model.fit(X_tfidf, target_train)

# predict
y_pred = mnb_model.predict(X_test_tfidf)

# evaluate
print("--- Test Set Performance ---")
print(classification_report(target_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     20362
           1       0.99      0.94      0.96     18356

    accuracy                           0.97     38718
   macro avg       0.97      0.97      0.97     38718
weighted avg       0.97      0.97      0.97     38718



In [ ]:
# overfitting check
y_train_pred = mnb_model.predict(X_tfidf)

print("--- Training Set Performance ---")
print(classification_report(target_train, y_train_pred))

--- Training Set Performance ---
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     81775
           1       1.00      0.97      0.98     73097

    accuracy                           0.98    154872
   macro avg       0.98      0.98      0.98    154872
weighted avg       0.98      0.98      0.98    154872



-> should not be overfittet, model performance just slightly(1%) better on train data than on test data